In [1]:
import numpy as np
import json
import copy

In [82]:
with open('sudoku_01.json') as f:
    d1 = json.load(f)
with open('sudoku_02.json') as f:
    d2 = json.load(f)
d1

[[5, 3, 0, 0, 7, 0, 0, 0, 0],
 [6, 0, 0, 1, 9, 5, 0, 0, 0],
 [0, 9, 8, 0, 0, 0, 0, 6, 0],
 [8, 0, 0, 0, 6, 0, 0, 0, 3],
 [4, 0, 0, 8, 0, 3, 0, 0, 1],
 [7, 0, 0, 0, 2, 0, 0, 0, 6],
 [0, 6, 0, 0, 0, 0, 2, 8, 0],
 [0, 0, 0, 4, 1, 9, 0, 0, 5],
 [0, 0, 0, 0, 8, 0, 0, 7, 9]]

In [142]:
class sudoku:
    def __init__(self, dim: int, base):
        self.dim = dim
        self.base = copy.deepcopy(base)
        self.tau, self.g = sudoku.create_tau(dim) 
        self.q = self.create_q()
        
    @staticmethod
    def complement(i,j, small_block):
        A = np.delete(small_block, i, 0)
        return np.delete(A, j, 1).flatten()
    
    @classmethod
    def create_tau(cls, dim):
        block = np.array([[f'{i}{j}' for j in range(9)] for i in range(9)])
        n = dim*dim
        tau = {}
        g = {}
        for i in range(n):
            for j in range(n):
                #small_block_pos
                i_ = int(i/dim) 
                j_ = int(j/dim)
                small_block = block[i_*dim:i_*dim+dim, j_*dim:j_*dim+dim]
                tau[f'{i}{j}'] = np.concatenate([np.delete(block[i,:], j),
                                                np.delete(block[:,j], i),
                                                cls.complement(i%dim,j%dim, small_block)])
                g[f'{i}{j}'] = {}
                for near in tau[f'{i}{j}']:
                    g[f'{i}{j}'][near] = np.ones((n,n), dtype=bool)
                    np.fill_diagonal(g[f'{i}{j}'][near], False)
        return tau, g
    
    def create_q(self):
        n = self.dim**2
        q = {}
        for i, v in enumerate(self.base):
            for j, e in enumerate(v):
                if e:
                    q[f'{i}{j}'] = np.zeros(n, dtype=bool)
                    q[f'{i}{j}'][e-1] = True
                else:
                    q[f'{i}{j}'] = np.ones(n, dtype=bool)
        return q
    
    def cut_g(self, g, q):
        same = True
        for t, neighbours in g.items():
            for t_, table in neighbours.items():
                Q_, Q = np.meshgrid(q[t_], q[t])
#                 print(table)
#                 print(q[t])
#                 print(q[t_])
                _new = table*Q_*Q
                if (g[t][t_] != _new).any() and same:
                    same = False
                g[t][t_] = table*Q_*Q
#                 print(table*Q_*Q)
        return same
                
    def cut_q(self, q, g, base):
        same = True
        max_amounts = []
        for t, states in q.items():
            arr = [states]
            for table in self.g[t].values():
                arr.append(table.any(axis=1))
            _new = np.array(arr).all(axis=0)
            if same and (q[t] != _new).any():
                same = False
            q[t] = np.array(arr).all(axis=0)

            amount = np.sum(q[t].astype(int))
            if amount == 0:
                print('0 STATES')
                return same, np.array([]), False
            elif amount == 1:
                i,j = t
                if base[int(i)][int(j)] == 0:
                    base[int(i)][int(j)] = list(q[t]).index(True)+1
                    print(np.asarray(base), t, list(q[t]).index(True)+1)
            else: 
                max_amounts.append([t, amount])
        return same, np.array(max_amounts), True
    
    def solve_case(self, base, g, q):
        g1 = copy.deepcopy(g)
        q1 = copy.deepcopy(q)
        solvable = True
#         print(base)
        while solvable:
            same_g = self.cut_g(g, q)
            same_q, amounts, solvable = self.cut_q(q, g, base)
            if not solvable:
                print('NOT SOLVABLE')
                return solvable, amounts, False 
            
            if same_q and same_g:
                print('same')
                # we need to figure out if it's the end
                if len(amounts):
                    return solvable, amounts, False
                else:
                    return solvable, amounts, True
    
    def solve_sudoku(self):
        solvable, amounts, solved = self.solve_case(self.base, self.g, self.q)        
        while solvable:
            if solved:
                return solved, self.base
            else:
                # we need to fix state in a node
                t, k = amounts[np.argsort(amounts[:,1])][0]
                pos1, pos2 = t
                available = np.arange(self.dim**2)[np.where(self.q[t])]
                for state in available:
                    print(f'Fixing state {state+1} in position {t}')
                    g0 = copy.deepcopy(self.g)
                    q0 = copy.deepcopy(self.q)
                    q0[t] = np.zeros(self.dim**2, dtype=bool)
                    q0[t][state] = True
                    base0 = copy.deepcopy(self.base)
                    base0[int(pos1)][int(pos2)] = state+1
                    print(np.asarray(base0), t, state+1)
                    solvable1, amounts1, solved1 = self.solve_case(base0, g0, q0)
                    if not solvable1:
                        continue
                    else:
                        print('solvable')
                        break
                solvable = solvable1
                self.base = base0
                self.g = g0
                self.q = q0
                amounts = amounts1
                solved = solved1
        return solved, self.base
    
    def solve(self):
        solved, base = self.solve_sudoku()
        if not solved:
            print("No solution found")
        else:
            print('Found solution')
            return base
            

In [143]:
d1

[[5, 3, 0, 0, 7, 0, 0, 0, 0],
 [6, 0, 0, 1, 9, 5, 0, 0, 0],
 [0, 9, 8, 0, 0, 0, 0, 6, 0],
 [8, 0, 0, 0, 6, 0, 0, 0, 3],
 [4, 0, 0, 8, 0, 3, 0, 0, 1],
 [7, 0, 0, 0, 2, 0, 0, 0, 6],
 [0, 6, 0, 0, 0, 0, 2, 8, 0],
 [0, 0, 0, 4, 1, 9, 0, 0, 5],
 [0, 0, 0, 0, 8, 0, 0, 7, 9]]

In [144]:
d2

[[8, 1, 0, 0, 3, 0, 0, 2, 7],
 [0, 6, 2, 0, 0, 0, 0, 9, 0],
 [0, 7, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 6, 0, 0, 1, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 4],
 [0, 0, 8, 0, 0, 5, 0, 7, 0],
 [0, 0, 0, 0, 0, 0, 0, 8, 0],
 [0, 0, 0, 0, 1, 0, 7, 5, 0],
 [0, 0, 0, 0, 7, 0, 0, 4, 2]]

In [145]:
s = sudoku(3, d2)

In [146]:
sol = s.solve()
sol

[[8 1 0 0 3 0 0 2 7]
 [0 6 2 0 0 0 0 9 0]
 [0 7 0 0 0 0 0 0 0]
 [0 0 0 6 0 0 1 3 0]
 [0 0 0 0 0 0 0 0 4]
 [0 0 8 0 0 5 0 7 0]
 [0 0 0 0 0 0 0 8 0]
 [0 0 0 0 1 0 7 5 0]
 [0 0 0 0 7 0 0 4 2]] 37 3
[[8 1 0 0 3 0 0 2 7]
 [0 6 2 0 0 0 0 9 0]
 [0 7 0 0 0 0 0 0 0]
 [0 0 0 6 0 0 1 3 0]
 [0 0 0 0 0 0 0 6 4]
 [0 0 8 0 0 5 0 7 0]
 [0 0 0 0 0 0 0 8 0]
 [0 0 0 0 1 0 7 5 0]
 [0 0 0 0 7 0 0 4 2]] 47 6
[[8 1 0 0 3 0 0 2 7]
 [0 6 2 0 0 0 0 9 0]
 [0 7 0 0 0 0 0 1 0]
 [0 0 0 6 0 0 1 3 0]
 [0 0 0 0 0 0 0 6 4]
 [0 0 8 0 0 5 0 7 0]
 [0 0 0 0 0 0 0 8 0]
 [0 0 0 0 1 0 7 5 0]
 [0 0 0 0 7 0 0 4 2]] 27 1
[[8 1 0 0 3 0 0 2 7]
 [0 6 2 0 0 0 0 9 0]
 [0 7 0 0 0 0 0 1 0]
 [0 0 0 6 0 0 1 3 0]
 [0 0 0 0 0 0 0 6 4]
 [0 0 8 0 0 5 0 7 9]
 [0 0 0 0 0 0 0 8 0]
 [0 0 0 0 1 0 7 5 0]
 [0 0 0 0 7 0 0 4 2]] 58 9
[[8 1 0 0 3 0 0 2 7]
 [0 6 2 0 0 0 0 9 0]
 [0 7 0 0 0 0 0 1 0]
 [0 0 0 6 0 0 1 3 0]
 [0 0 0 0 0 0 0 6 4]
 [0 0 8 0 0 5 2 7 9]
 [0 0 0 0 0 0 0 8 0]
 [0 0 0 0 1 0 7 5 0]
 [0 0 0 0 7 0 0 4 2]] 56 2
[[8 1 0 0 3 0 0 2 7]
 [0 

same
solvable
Fixing state 2 in position 25
[[8 1 5 9 3 6 4 2 7]
 [4 6 2 7 8 1 3 9 5]
 [3 7 9 2 5 2 8 1 6]
 [0 0 4 6 2 7 1 3 8]
 [0 2 0 0 9 0 5 6 4]
 [6 3 8 1 4 5 2 7 9]
 [0 0 0 0 6 0 9 8 1]
 [0 0 0 0 1 0 7 5 3]
 [0 0 0 0 7 0 6 4 2]] 25 2
0 STATES
NOT SOLVABLE
Fixing state 4 in position 25
[[8 1 5 9 3 6 4 2 7]
 [4 6 2 7 8 1 3 9 5]
 [3 7 9 2 5 4 8 1 6]
 [0 0 4 6 2 7 1 3 8]
 [0 2 0 0 9 0 5 6 4]
 [6 3 8 1 4 5 2 7 9]
 [0 0 0 0 6 0 9 8 1]
 [0 0 0 0 1 0 7 5 3]
 [0 0 0 0 7 0 6 4 2]] 25 4
[[8 1 5 9 3 6 4 2 7]
 [4 6 2 7 8 1 3 9 5]
 [3 7 9 2 5 4 8 1 6]
 [0 0 4 6 2 7 1 3 8]
 [0 2 0 0 9 0 5 6 4]
 [6 3 8 1 4 5 2 7 9]
 [0 0 0 0 6 0 9 8 1]
 [0 0 6 0 1 0 7 5 3]
 [0 0 0 0 7 0 6 4 2]] 72 6
same
solvable
Fixing state 5 in position 30
[[8 1 5 9 3 6 4 2 7]
 [4 6 2 7 8 1 3 9 5]
 [3 7 9 2 5 4 8 1 6]
 [5 0 4 6 2 7 1 3 8]
 [0 2 0 0 9 0 5 6 4]
 [6 3 8 1 4 5 2 7 9]
 [0 0 0 0 6 0 9 8 1]
 [0 0 6 0 1 0 7 5 3]
 [0 0 0 0 7 0 6 4 2]] 30 5
same
solvable
Fixing state 5 in position 31
[[8 1 5 9 3 6 4 2 7]
 [4 6 2 7 8 1 3

[[8, 1, 5, 9, 3, 6, 4, 2, 7],
 [4, 6, 2, 7, 8, 1, 3, 9, 5],
 [3, 7, 9, 2, 5, 4, 8, 1, 6],
 [5, 9, 4, 6, 2, 7, 1, 3, 8],
 [1, 2, 7, 3, 9, 8, 5, 6, 4],
 [6, 3, 8, 1, 4, 5, 2, 7, 9],
 [7, 4, 3, 5, 6, 2, 9, 8, 1],
 [2, 8, 6, 4, 1, 9, 7, 5, 3],
 [9, 5, 1, 8, 7, 3, 6, 4, 2]]